In [1]:
%run read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5230c441-0590-4bc1-8e9c-ba4aad13b691;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (41ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (23ms)
:: resolution report :: resolve 668ms :: artifacts dl 68ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/03/31 01:41:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/31 01:41:27 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
%run trace_encode.ipynb

In [3]:
df_all = de_dupe()

 1:17AM UTC on Mar 30, 2023 --- read time: 4.651973247528076 seconds ---


In [10]:
df_all.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- image_path_ext: string (nullable = true)
 |-- parent_path_ext: string (nullable = true)
 |-- new_path_ext: string

In [10]:
tot = df_all.count()

In [17]:
#check days and malicious counts
df_all.groupBy("event_day", "malicious") \
              .count().cache().show()

+---------+---------+-------+
|event_day|malicious|  count|
+---------+---------+-------+
|       25|        1|    343|
|       23|        0|1206627|
|       24|        1|   4362|
|       23|        1|   3307|
|       24|        0|1503638|
|       25|        0|1038587|
+---------+---------+-------+



In [4]:
df_all = df_all.cache()

In [ ]:
#how many image paths are there
df_all.groupBy('image_path_ext','parent_path_ext').count().show()

+--------------+---------------+-------+
|image_path_ext|parent_path_ext|  count|
+--------------+---------------+-------+
|              |               |  27514|
|              |           .exe|  12340|
|          null|           null| 334817|
|          .exe|           .exe| 899444|
|          .exe|           null|1875351|
|              |           null| 596814|
|          .exe|               |  10580|
|          .com|           .exe|      1|
|          .com|           null|      3|
+--------------+---------------+-------+



In [94]:
list_img = ['None','empty','.exe','.com']
list_par_img = ['None','empty','.exe']

In [20]:
#how many new paths are there
df_all.select('new_path_ext').distinct().count()

24

In [22]:
#what are the new path counts
df_all.groupBy('new_path_ext').count().show()

+------------+-------+
|new_path_ext|  count|
+------------+-------+
|        .log|    954|
|        null|3734496|
|        .dat|    500|
|        .etl|  19061|
|        .bak|    197|
|        .tmp|    343|
|    .jsonlz4|    156|
|        .odl|    134|
|            |    565|
|        .cvr|    241|
|        .jtx|     15|
|     .mozlz4|     16|
|         .js|     19|
|       .json|     72|
|        .yml|     65|
|       .pptx|      2|
|        .dll|      4|
|        .bin|      8|
|        .zip|      1|
|        .ini|      3|
+------------+-------+
only showing top 20 rows



In [21]:
#how many file paths are there
df_all.select('file_path_ext').distinct().count()

225

In [16]:
#how many file paths are there
ext_ratios = df_all.groupBy('file_path_ext').count().sort(col('count').desc()).withColumn('perc_of_count_total', (col('count') / tot) * 100).limit(20)


In [17]:
ext_ratios.show()

+-------------+-------+-------------------+
|file_path_ext|  count|perc_of_count_total|
+-------------+-------+-------------------+
|         null|1294015|  34.44402033185124|
|             |1093356| 29.102890070015842|
|         .dll| 421606| 11.222285395478783|
|         .pyc| 271542|  7.227890070015842|
|         .bat|  48676| 1.2956550995724092|
|          .pf|  46930| 1.2491801672884617|
|         .ini|  41568| 1.1064547452343232|
|         .exe|  37894| 1.0086604146437028|
|         .tmp|  35275| 0.9389480162177817|
|         .cat|  33596| 0.8942564862608814|
|         .dat|  29786|  0.792842115125807|
|         .etl|  28047| 0.7465535084581183|
|         .xml|  26723| 0.7113113490400504|
|         .log|  25180| 0.6702398596276044|
|         .mum|  23850| 0.6348379925384575|
|        .evtx|  22010|  0.585860973407608|
|     .sbstore|  18984| 0.5053150712935044|
|        .pset|  18841| 0.5015087051327917|
|         .egg|  14665| 0.3903521660619069|
|         .mui|  13433| 0.357558

In [5]:
#create a list of top 20 file extensions
file_ext = list(df_all.groupBy('file_path_ext').count().sort(col('count').desc()).limit(9).toPandas()['file_path_ext'])

In [6]:
df_23 = df_all.where(col('event_day') == 23).cache()

In [8]:
df_23_1516 = df_23.where((col('event_hour') == 16) | (col('event_hour') == 15)).cache()

In [9]:
df_23_1516.count()

329305

In [7]:
df_23_15 = df_23.where((col('event_hour') == 15)).cache()

In [8]:
df_23_15.count()

168873

In [20]:
#now create graph for event traces of 2 events, 4 events, and 6 events
start_time = time.time()
    
#create trace matrix from malicious events for speed. 
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df_23_15.selectExpr('actorID as id').distinct()
dst_vertices = df_23_15.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_23_15.selectExpr('objectID as src', 'actorID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                          'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c)") #; (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")
print("found connections: "+ str(time.time() - start_time))
#create paths and count
# filter paths to only those where all edges are connected
cp_2315_2 = motifs6.filter('''e1.timestamp <= e2.timestamp''').cache()# and e2.timestamp <= e3.timestamp and 
                                    #e3.timestamp <= e4.timestamp and e4.timestamp <= e5.timestamp and 
                                    #e5.timestamp <= e6.timestamp''').cache()

print("event traces: "+str(cp_2315_2.count()))

print("create graph: "+ str(time.time() - start_time))

found connections: 0.1743764877319336


event traces: 358853
create graph: 13.880214214324951


In [13]:
#now create graph for event traces of 2 events, 3 events
start_time = time.time()
    
#create trace matrix from malicious events for speed. 
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df_23_15.selectExpr('actorID as id').distinct()
dst_vertices = df_23_15.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_23_15.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                          'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)") #; (d)-[e4]->(e)") #; (e)-[e5]->(f); (f)-[e6]->(g)")
print("found connections: "+ str(time.time() - start_time))
#create paths and count
# filter paths to only those where all edges are connected
cp_2315_3 = motifs6.filter('''e1.timestamp <= e2.timestamp and e2.timestamp <= e3.timestamp''')# and 
                                    #e3.timestamp <= e4.timestamp''').cache()# and e4.timestamp <= e5.timestamp and 
                                    #e5.timestamp <= e6.timestamp''').cache()

print("event traces: "+str(cp_2315_3.count()))

print("create graph: "+ str(time.time() - start_time))

found connections: 0.4217662811279297


event traces: 257923798
create graph: 1037.19176030159


In [18]:
#now create graph for event traces of 2 events, 3 events
start_time = time.time()
    
#create trace matrix from malicious events for speed. 
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df_23_15.selectExpr('actorID as id').distinct()
dst_vertices = df_23_15.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_23_15.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                          'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)") #; (d)-[e4]->(e)") #; (e)-[e5]->(f); (f)-[e6]->(g)")
print("found connections: "+ str(time.time() - start_time))
#create paths and count
# filter paths to only those where all edges are connected
cp_2315_3 = motifs6.filter('''e1.timestamp <= e2.timestamp and e2.timestamp <= e3.timestamp''')# and 
                                    #e3.timestamp <= e4.timestamp''').cache()# and e4.timestamp <= e5.timestamp and 
                                    #e5.timestamp <= e6.timestamp''').cache()

print("event traces: "+str(cp_2315_3.count()))

print("create graph: "+ str(time.time() - start_time))

found connections: 0.24465680122375488


event traces: 1108974
create graph: 12.938036918640137


In [170]:
#now create graph for event traces of 2 events, 3 events
start_time = time.time()
    
#create trace matrix from malicious events for speed. 
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df_23_15.selectExpr('objectID as id').distinct()
dst_vertices = df_23_15.selectExpr('actorID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_23_15.selectExpr('objectID as src', 'actorID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                          'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)") #; (d)-[e4]->(e)") #; (e)-[e5]->(f); (f)-[e6]->(g)")
print("found connections: "+ str(time.time() - start_time))
#create paths and count
# filter paths to only those where all edges are connected
cp_2315_3oa = motifs6.filter('''e1.timestamp <= e2.timestamp and e2.timestamp <= e3.timestamp''')# and 
                                    #e3.timestamp <= e4.timestamp''').cache()# and e4.timestamp <= e5.timestamp and 
                                    #e5.timestamp <= e6.timestamp''').cache()

print("event traces: "+str(cp_2315_3oa.count()))

print("create graph: "+ str(time.time() - start_time))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


found connections: 0.21571063995361328


event traces: 466933
create graph: 12.180915594100952


In [171]:
df_transp = cp_2315_3oa.withColumn("Trace", (monotonically_increasing_id() + 1))
df_transp = df_transp.select("Trace", 
                             *[col for col in df_transp.columns if col != "Trace"])

#drop all vertices
df_transp = df_transp.drop('a','b','c','d')

#transpose rows 
stacked_df = df_transp.selectExpr(
    "Trace", 
    "posexplode(array(e1, e2, e3)) as (pos, col)"
).select(
    "Trace", 
    expr('''CASE pos 
    WHEN 0 THEN 'e1' 
    WHEN 1 THEN 'e2'
    ELSE 'e3' END''').alias("event"),
    "col"
).orderBy("Trace","event")

#explode columns
df_onehot_oa = stacked_df.select(*stacked_df.columns, "col.*").drop('col').cache()

df_transp = cp_2315_3.withColumn("Trace", (monotonically_increasing_id() + 1))
df_transp = df_transp.select("Trace", 
                             *[col for col in df_transp.columns if col != "Trace"])

#drop all vertices
df_transp = df_transp.drop('a','b','c','d')

#transpose rows 
stacked_df = df_transp.selectExpr(
    "Trace", 
    "posexplode(array(e1, e2, e3)) as (pos, col)"
).select(
    "Trace", 
    expr('''CASE pos 
    WHEN 0 THEN 'e1' 
    WHEN 1 THEN 'e2'
    ELSE 'e3' END''').alias("event"),
    "col"
).orderBy("Trace","event")

#explode columns
df_onehot = stacked_df.select(*stacked_df.columns, "col.*").drop('col').cache()

In [172]:
df_onehot_oa.limit(6).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,image_path,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious
0,1,e1,2b2a9a10-bcc0-413e-b6f1-8237c4579166,9274fa54-92f9-4b36-84c8-4b103c3e931a,2019-09-23 15:04:40.021,FILE,READ,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Python27\python.exe,None,None,\Device\HarddiskVolume1\Windows\SysWOW64\adsld...,None,None,None,None,0
1,1,e2,9274fa54-92f9-4b36-84c8-4b103c3e931a,97fc76f2-2e7c-4c21-8a82-ae191c19a33f,2019-09-23 15:09:54.088,PROCESS,OPEN,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\conho...,\Device\HarddiskVolume1\Windows\system32\conho...,None,None,None,None,None,None,0
2,1,e3,97fc76f2-2e7c-4c21-8a82-ae191c19a33f,43a5e405-a143-4f3a-8954-0df7cc9abd87,2019-09-23 15:10:09.743,PROCESS,OPEN,SysClient0321.systemia.com,None,None,%SystemRoot%\system32\csrss.exe,%SystemRoot%\system32\csrss.exe,None,None,None,None,None,None,0
3,2,e1,2b2a9a10-bcc0-413e-b6f1-8237c4579166,9274fa54-92f9-4b36-84c8-4b103c3e931a,2019-09-23 15:04:40.021,FILE,READ,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Python27\python.exe,None,None,\Device\HarddiskVolume1\Windows\SysWOW64\adsld...,None,None,None,None,0
4,2,e2,9274fa54-92f9-4b36-84c8-4b103c3e931a,97fc76f2-2e7c-4c21-8a82-ae191c19a33f,2019-09-23 15:09:54.088,PROCESS,OPEN,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\conho...,\Device\HarddiskVolume1\Windows\system32\conho...,None,None,None,None,None,None,0
5,2,e3,97fc76f2-2e7c-4c21-8a82-ae191c19a33f,a059d617-993c-4772-8a96-803a2fc0eef2,2019-09-23 15:10:09.721,PROCESS,OPEN,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\svcho...,\Device\HarddiskVolume1\Windows\system32\svcho...,None,None,None,None,None,None,0


In [24]:
df_onehot.limit(6).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,image_path,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious
0,1,e1,addd4dd5-e6b8-471e-94b7-c391ea02da0c,27c3c561-4929-4241-97c2-e7fd688a7951,2019-09-23 15:17:56.543,PROCESS,CREATE,SysClient0358.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\cmd.exe,\Device\HarddiskVolume1\Python27\python.exe,None,None,None,None,None,None,0
1,1,e2,27c3c561-4929-4241-97c2-e7fd688a7951,06ca5760-85df-4718-b2c3-77f76c3a3e2d,2019-09-23 15:17:56.573,PROCESS,CREATE,SysClient0358.systemia.com,None,None,taskkill.exe,\Device\HarddiskVolume1\Windows\system32\cmd.exe,None,None,None,None,None,None,0
2,1,e3,06ca5760-85df-4718-b2c3-77f76c3a3e2d,52fa575b-8385-4ded-b607-93b404eb367d,2019-09-23 15:17:56.692,FILE,READ,SysClient0358.systemia.com,None,None,taskkill.exe,None,None,\Device\HarddiskVolume1\Windows\SysWOW64\Winst...,None,None,None,None,0
3,2,e1,addd4dd5-e6b8-471e-94b7-c391ea02da0c,27c3c561-4929-4241-97c2-e7fd688a7951,2019-09-23 15:17:56.543,PROCESS,CREATE,SysClient0358.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\cmd.exe,\Device\HarddiskVolume1\Python27\python.exe,None,None,None,None,None,None,0
4,2,e2,27c3c561-4929-4241-97c2-e7fd688a7951,06ca5760-85df-4718-b2c3-77f76c3a3e2d,2019-09-23 15:17:56.573,PROCESS,CREATE,SysClient0358.systemia.com,None,None,taskkill.exe,\Device\HarddiskVolume1\Windows\system32\cmd.exe,None,None,None,None,None,None,0
5,2,e3,06ca5760-85df-4718-b2c3-77f76c3a3e2d,976fa881-0178-4a57-b129-c97d5bec03b8,2019-09-23 15:17:56.652,FILE,READ,SysClient0358.systemia.com,None,None,taskkill.exe,None,None,\Device\HarddiskVolume1\Windows\SysWOW64\netut...,None,None,None,None,0


In [17]:
#write both samples to parquet
s3_url_trusted = "s3a://sapient-bucket-trusted/"
cp_2315_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test/2315_2")

In [15]:
#write both samples to parquet
s3_url_trusted = "s3a://sapient-bucket-trusted/"
cp_2315_4.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test/2315_4")

ERROR:root:KeyboardInterrupt while sending command.             (18 + 16) / 200]
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

## So now let's work on filtering

In [203]:
#create the necessary file extension filtering
df = df_onehot_oa.withColumn("image_path_ext", regexp_extract("image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("parent_path_ext", regexp_extract("parent_image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("new_path_ext", regexp_extract("new_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("file_path_ext", regexp_extract("file_path", "\.[0-9a-z]+$", 0)).cache()

In [32]:
tot = df.count()

In [34]:
df.groupBy('file_path_ext').count().sort(col('count').desc()).withColumn('perc_of_count_total', (col('count') / tot) * 100).limit(20).show()

+-------------+-------+-------------------+
|file_path_ext|  count|perc_of_count_total|
+-------------+-------+-------------------+
|         null|1017821|   72.6600318818046|
|             | 151294| 10.800550257388819|
|         .dll| 136872|  9.770994982149475|
|         .pyc|  20031| 1.4299696102010353|
|         .cat|  13868| 0.9900064177658608|
|         .ini|   9862| 0.7040267732915286|
|          .pf|   4175| 0.2980441876386263|
|         .dat|   3756| 0.2681326871307018|
|         .tmp|   3466| 0.2474302166120907|
|         .exe|   2504|0.17875512475380123|
|         .egg|   2403|0.17154495398697456|
|     .sbstore|   2234| 0.1594804108226805|
|        .pset|   2210|0.15776710291769197|
|         .mui|   2117| 0.1511280347858615|
|         .txt|   1769| 0.1262850701635281|
|          .js|   1759|0.12557119186978288|
|      .sqlite|   1584|0.11307832172924168|
|         .etl|   1529|  0.109151991113643|
|          .db|   1496|0.10679619274428379|
|         .log|   1452|0.1036551

In [40]:
file_ext = [str(i) for i in file_ext]

In [99]:
file_ext = ['None', 'empty', '.dll', '.pyc', '.bat', '.pf', '.ini', '.exe', '.tmp']

In [100]:
df = df.withColumn("file_path_ext", when(col("file_path_ext").isNull(),"None").otherwise(col("file_path_ext"))) \
.withColumn("file_path_ext", when(col("file_path_ext") == '', 'empty').otherwise(col("file_path_ext"))) \
.withColumn("file_path_ext", when(col("file_path_ext").isin(file_ext), col('file_path_ext')).otherwise('other')).cache()

In [101]:
list_img = ['None','empty','.exe','.com']
list_par_img = ['None','empty','.exe']

df = df.withColumn("image_path_ext", when(col("image_path_ext").isNull(),"None").otherwise(col("image_path_ext"))) \
.withColumn("image_path_ext", when(col("image_path_ext") == '', 'empty').otherwise(col("image_path_ext"))) \
.withColumn("image_path_ext", when(col("image_path_ext").isin(list_img), col('image_path_ext')).otherwise('other')).cache()

In [102]:
df = df.withColumn("parent_path_ext", when(col("parent_path_ext").isNull(),"None").otherwise(col("parent_path_ext"))) \
.withColumn("parent_path_ext", when(col("parent_path_ext") == '', 'empty').otherwise(col("parent_path_ext"))).cache()

In [103]:
df.select('image_path_ext').distinct().show()

+--------------+
|image_path_ext|
+--------------+
|          None|
|          .exe|
|         empty|
+--------------+



In [104]:
df.select('parent_path_ext').distinct().show()

+---------------+
|parent_path_ext|
+---------------+
|           None|
|           .exe|
|          empty|
+---------------+



In [48]:
#now filter the file paths based on the 9 we selected above
df = df.withColumn("file_path_ext",
                             when(col("file_path_ext").isin(file_ext), 
                                  col('file_path_ext')).otherwise('other')).cache()

In [105]:
#check results
df.select('file_path_ext').distinct().show()

+-------------+
|file_path_ext|
+-------------+
|         .ini|
|         None|
|          .pf|
|        other|
|         .pyc|
|         .tmp|
|         .exe|
|         .dll|
|        empty|
|         .bat|
+-------------+



In [92]:
df.select('image_path_ext').distinct().show()

+--------------+
|image_path_ext|
+--------------+
|          null|
|          .exe|
|              |
+--------------+



In [93]:
df.select('parent_path_ext').distinct().show()

+---------------+
|parent_path_ext|
+---------------+
|           null|
|           .exe|
|               |
+---------------+



In [63]:
#now work on labeling entire malicious trace
# create a sample dataframe that aligns closer with what you see above. 
#I think the trick is going to be partitioning based on object action and timestamp.
#if those three things are the same then pull the first one out
data = [("A", "B",1, 1,'process','create',0), 
        ("B", "C",1, 2,'process','create',0), 
        ("C", "D",1, 3,'process','open',0),
        ("D", "E",2,4,'process','open',1),
        ("E", "F",2, 5,'file','read',0),
       ("F", "G",2, 6,'process','create',0), 
        ("G", "H",3, 7,'process','create',0), 
        ("H", "I",3, 8,'process','open',1),
        ("I", "J" ,3,9,'process','open',0),
        ("J", "K",3, 10,'file','read',1),
       ("K", "L",4, 8,'process','open',1),
        ("M", "N",4,9,'process','open',1),
        ("O", "P",4, 10,'file','read',1)]
columns = ["src", "dst","Trace", "time","object","action",'malicious']
df_samp = spark.createDataFrame(data, columns)

In [64]:
#create a window method to label traces as malicious given one event
w = Window.partitionBy("Trace")#.orderBy("time")

# Apply the conditions to the window using the when function
df_samp = df_samp.withColumn('mal_trace', when(sum('malicious').over(w) > 0, 1).otherwise(0))
#df_samp = df_samp.withColumn('mal_trace', sum('mal_trace').over(w))

In [65]:
df_samp.show()

+---+---+-----+----+-------+------+---------+---------+
|src|dst|Trace|time| object|action|malicious|mal_trace|
+---+---+-----+----+-------+------+---------+---------+
|  A|  B|    1|   1|process|create|        0|        0|
|  B|  C|    1|   2|process|create|        0|        0|
|  C|  D|    1|   3|process|  open|        0|        0|
|  D|  E|    2|   4|process|  open|        1|        1|
|  E|  F|    2|   5|   file|  read|        0|        1|
|  F|  G|    2|   6|process|create|        0|        1|
|  G|  H|    3|   7|process|create|        0|        1|
|  H|  I|    3|   8|process|  open|        1|        1|
|  I|  J|    3|   9|process|  open|        0|        1|
|  J|  K|    3|  10|   file|  read|        1|        1|
|  K|  L|    4|   8|process|  open|        1|        1|
|  M|  N|    4|   9|process|  open|        1|        1|
|  O|  P|    4|  10|   file|  read|        1|        1|
+---+---+-----+----+-------+------+---------+---------+



In [74]:
#calculate malicious trace then check schema and how many were made 
w = Window.partitionBy("Trace")
df = df.withColumn('mal_trace', when(sum('malicious').over(w) > 0, 1).otherwise(0)).cache()

In [75]:
df.printSchema()

root
 |-- Trace: long (nullable = false)
 |-- event: string (nullable = false)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- image_path_ext: string (nullable = true)
 |-- parent_path_ext: string (nullable = true)
 |-- new_path_ext: string (nullable = true)
 |-- file_path_ext: string (nullable = true)
 |-- mal_trace: integer (nullable =

In [76]:
df.groupby('mal_trace').count().show()

+---------+-------+
|mal_trace|  count|
+---------+-------+
|        1|  26502|
|        0|1374297|
+---------+-------+



In [106]:
for c in ['file_path_ext','image_path_ext','parent_path_ext','object','action']:
    df, dict_mapping, cols_sparse = oneHotCol(df, c, {}, [])

In [107]:
df.printSchema()

root
 |-- Trace: long (nullable = false)
 |-- event: string (nullable = false)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- new_path_ext: string (nullable = true)
 |-- file_path_ext_sparse: vector (nullable = true)
 |-- image_path_ext_sparse: vector (nullable = true)
 |-- parent_path_ext_sparse: vector (nullable = true)
 |-- object_sparse: vector (nullable = true)
 |-- action_sparse: vector (nullable = true)


In [108]:
df.limit(2).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,hostname,user_name,privileges,image_path,parent_image_path,...,logon_id,requesting_domain,requesting_user,malicious,new_path_ext,file_path_ext_sparse,image_path_ext_sparse,parent_path_ext_sparse,object_sparse,action_sparse
0,1,e1,2b2a9a10-bcc0-413e-b6f1-8237c4579166,9274fa54-92f9-4b36-84c8-4b103c3e931a,2019-09-23 15:04:40.021,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Python27\python.exe,None,...,None,None,None,0,None,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,e2,9274fa54-92f9-4b36-84c8-4b103c3e931a,97fc76f2-2e7c-4c21-8a82-ae191c19a33f,2019-09-23 15:09:54.088,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\conho...,\Device\HarddiskVolume1\Windows\system32\conho...,...,None,None,None,0,None,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [109]:
#calculate malicious trace then check schema and how many were made 
w = Window.partitionBy("Trace")
df = df.withColumn('mal_trace', when(sum('malicious').over(w) > 0, 1).otherwise(0)).cache()

In [205]:
#now adjust the timediff bins
df = ts_diff(df).cache()

In [111]:
df.printSchema()

root
 |-- Trace: long (nullable = false)
 |-- event: string (nullable = false)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- new_path_ext: string (nullable = true)
 |-- file_path_ext_sparse: vector (nullable = true)
 |-- image_path_ext_sparse: vector (nullable = true)
 |-- parent_path_ext_sparse: vector (nullable = true)
 |-- object_sparse: vector (nullable = true)
 |-- action_sparse: vector (nullable = true)


In [137]:
#develop quantiles for timestamp diff
bins = df.approxQuantile("timestamp_difference", [0.2, 0.4, .6, .8], 0)

In [214]:
# create a Bucketizer instance
#bins = df.approxQuantile("timestamp_difference", [0.2, 0.4, .6, .8], 0)
#bins_2 = [float("-inf")] + bins + [float("inf")]
#bucketizer = Bucketizer(splits=bins_2, inputCol="timestamp_difference", outputCol="timestamp_bins_3")
# transform the DataFrame
#df = bucketizer.transform(df)
df_2 = df.withColumn("bin_column", when(col("timestamp_difference") <= bins[0], "1") \
              .when((col("timestamp_difference") > bins[0]) & (col('timestamp_difference') <= bins[1]), "2") \
                .when((col("timestamp_difference") > bins[1]) & (col('timestamp_difference') <= bins[2]), "3") \
                     .when((col("timestamp_difference") > bins[2]) & (col('timestamp_difference') <= bins[3]), "4") \
                     .when((col("timestamp_difference") > bins[3]), 5).otherwise('first'))
                     
              #((df["timestamp_difference"] > percentiles[0]) & (df["timestamp_difference"] <= percentiles[1]), "2")).otherwise("no"))
    #when((df["timestamp_difference"] > percentiles[1]) & (df["timestamp_difference"] <= percentiles[2]), "3"), 
    #when((df["timestamp_difference"] > percentiles[2]) & (df["timestamp_difference"] <= percentiles[3]), "4"),
    #when(df["timestamp_difference"] > percentiles[3], "5").otherwise("First"))

## bins

In [157]:
df = bin_it(ts_diff(df))

In [158]:
df.limit(2).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,...,direction,logon_id,requesting_domain,requesting_user,malicious,image_path_ext,parent_path_ext,new_path_ext,file_path_ext,timestamp_bins
0,26,e1,1c5f0325-2863-46ab-8677-b3355a5bc649,9274fa54-92f9-4b36-84c8-4b103c3e931a,2019-09-23 15:09:52.395,FILE,READ,SysClient0321.systemia.com,None,None,...,None,None,None,None,0,.exe,None,None,.pyc,NaN
1,26,e2,9274fa54-92f9-4b36-84c8-4b103c3e931a,97fc76f2-2e7c-4c21-8a82-ae191c19a33f,2019-09-23 15:09:54.088,PROCESS,OPEN,SysClient0321.systemia.com,None,None,...,None,None,None,None,0,.exe,.exe,None,None,0.0


In [120]:
df.printSchema()

root
 |-- Trace: long (nullable = false)
 |-- event: string (nullable = false)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- new_path_ext: string (nullable = true)
 |-- file_path_ext_sparse: vector (nullable = true)
 |-- image_path_ext_sparse: vector (nullable = true)
 |-- parent_path_ext_sparse: vector (nullable = true)
 |-- object_sparse: vector (nullable = true)
 |-- action_sparse: vector (nullable = true)


In [122]:
type(bins)

list

In [121]:
df.count()

23/03/30 04:33:06 ERROR Executor: Exception in task 1.0 in stage 851.0 (TID 44568)
org.apache.spark.SparkException: Failed to execute user defined function (Bucketizer$$Lambda$7033/469333003: (double) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:177)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.columnar.DefaultCachedBatchSerializer$$anon$1.next(InMemoryRelation.scala:87)
	at org.apache.spark.sql.execution.columnar.DefaultCachedBatchSerializer$$anon$1.next(InMemoryRelation.scala:79

Py4JJavaError: An error occurred while calling o2307.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 851.0 failed 1 times, most recent failure: Lost task 0.0 in stage 851.0 (TID 44567) (ip-172-16-41-81.us-west-2.compute.internal executor driver): org.apache.spark.SparkException: Failed to execute user defined function (Bucketizer$$Lambda$7033/469333003: (double) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:177)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.s

## Now run your function and see what happens, don't forget to update for trace 3/4

In [271]:
%run trace_encode.ipynb

In [251]:
df_23 = de_dupe()

 6:15AM UTC on Mar 30, 2023 --- read time: 0.2825961112976074 seconds ---


In [252]:
df_2315 = df_23.where((col('event_hour') == 15))

In [253]:
df_2315.select('file_path_ext').distinct().show()

+-------------+
|file_path_ext|
+-------------+
|         .ini|
|         None|
|          .pf|
|        other|
|         .pyc|
|         .tmp|
|         .bat|
|         .exe|
|         .dll|
|        empty|
+-------------+



In [185]:
df_2315.count()

168873

In [216]:
df_graph = create_graph3(df_2315)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


found connections: 0.2606477737426758
event traces: 466933


In [221]:
df_onehot = transp_expl3(df_graph)

In [229]:
#calculate malicious trace then check schema and how many were made 
w = Window.partitionBy("Trace")
df_onehot = df_onehot.withColumn('mal_trace', when(sum('malicious').over(w) > 0, 1).otherwise(0)).cache()
    

In [230]:
#now develop time_diff bins
df_onehot = bin_it(df_onehot)

In [180]:
#now create graph for event traces of 2 events, 3 events
start_time = time.time()
    
#create trace matrix from malicious events for speed. 
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df_2315.selectExpr('objectID as id').distinct()
dst_vertices = df_2315.selectExpr('actorID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_2315.selectExpr('objectID as src', 'actorID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                          'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)") #; (d)-[e4]->(e)") #; (e)-[e5]->(f); (f)-[e6]->(g)")
print("found connections: "+ str(time.time() - start_time))
#create paths and count
# filter paths to only those where all edges are connected
cp_2315_3oa = motifs6.filter('''e1.timestamp <= e2.timestamp and e2.timestamp <= e3.timestamp''')# and 
                                    #e3.timestamp <= e4.timestamp''').cache()# and e4.timestamp <= e5.timestamp and 
                                    #e5.timestamp <= e6.timestamp''').cache()

print("event traces: "+str(cp_2315_3oa.count()))

print("create graph: "+ str(time.time() - start_time))

df_onehot = transp_expl3(cp_2315_3oa)

found connections: 0.2523486614227295
event traces: 466933
create graph: 0.7373178005218506


In [226]:
df_onehot.sort("Trace").limit(6).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,image_path,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious,bin_column
0,1,e3,9274fa54-92f9-4b36-84c8-4b103c3e931a,97fc76f2-2e7c-4c21-8a82-ae191c19a33f,2019-09-23 15:09:54.088,PROCESS,OPEN,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\conho...,\Device\HarddiskVolume1\Windows\system32\conho...,None,None,None,None,None,None,0,1
1,1,e2,1e4b229d-e3aa-457e-a351-c90b35ddde69,9274fa54-92f9-4b36-84c8-4b103c3e931a,2019-09-23 15:09:54.074,PROCESS,OPEN,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Python27\python.exe,\Device\HarddiskVolume1\Python27\python.exe,None,None,None,None,None,None,0,1
2,1,e1,df26cb51-6838-4d54-b63b-b32a0152638d,1e4b229d-e3aa-457e-a351-c90b35ddde69,2019-09-23 15:09:53.977,PROCESS,OPEN,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\cmd.exe,\Device\HarddiskVolume1\Windows\system32\cmd.exe,None,None,None,None,None,None,0,first
3,2,e1,23e70934-4ca4-400c-9935-f14884268370,1e4b229d-e3aa-457e-a351-c90b35ddde69,2019-09-23 15:09:54.020,FILE,READ,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\cmd.exe,None,None,\Device\HarddiskVolume1\Windows\SysWOW64\winbr...,None,None,None,None,0,first
4,2,e2,1e4b229d-e3aa-457e-a351-c90b35ddde69,9274fa54-92f9-4b36-84c8-4b103c3e931a,2019-09-23 15:09:54.074,PROCESS,OPEN,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Python27\python.exe,\Device\HarddiskVolume1\Python27\python.exe,None,None,None,None,None,None,0,1
5,2,e3,9274fa54-92f9-4b36-84c8-4b103c3e931a,97fc76f2-2e7c-4c21-8a82-ae191c19a33f,2019-09-23 15:09:54.088,PROCESS,OPEN,SysClient0321.systemia.com,None,None,\Device\HarddiskVolume1\Windows\system32\conho...,\Device\HarddiskVolume1\Windows\system32\conho...,None,None,None,None,None,None,0,1


In [231]:
df_onehot.printSchema()

root
 |-- Trace: long (nullable = false)
 |-- event: string (nullable = false)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- bin_column: string (nullable = false)
 |-- mal_trace: integer (nullable = false)



In [244]:
#for all columns to one hot, one hot, preserve mapping
for colm in ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action']+["timestamp_bins"]:
    print(colm)
    df_onehot, dict_mapping, list_sparse = oneHotCol(df_onehot,colm, {}, [])

file_path_ext


Py4JJavaError: An error occurred while calling o6324.fit.
: org.apache.spark.SparkException: Input column file_path_ext does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:293)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:290)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apache.

In [ ]:
num = colm+'_numeric'
sparse = colm+'_sparse'
indexer = StringIndexer(inputCol=colm, outputCol=num, handleInvalid="keep")
indexer_fitted = indexer.fit(df)
df_indexed = indexer_fitted.transform(df)

In [256]:
df_enc,dict_mapping = trace_encode(df_2315, 3,
                                   ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action'])

found connections: 0.3053398132324219


event traces: 466933
create graph: 16.650644540786743


transposed explode and bin: 34.64272904396057


one-hot time: 138.65559077262878


total elapsed time: 155.27439427375793


In [259]:
df_enc4,dict_mapping4 = trace_encode(df_2315, 4,
                                   ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action'])

found connections: 0.41779136657714844


event traces: 309859
create graph: 21.555825233459473


transposed explode and bin: 44.78338027000427


one-hot time: 179.72427415847778


total elapsed time: 201.0058970451355


### Read/Write examples below

In [268]:
df_enc.where((col('mal_trace')==1)&(col('malicious')==1)).count()

2338

In [269]:
#this math checks out 
df_enc.count()

466933

In [272]:
test = create_graph6(df_2315)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


found connections: 0.7146964073181152


event traces: 52796


In [273]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
df_enc.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/encoded/sample3")

In [274]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
df_enc4.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/encoded/sample4")

### Create real data with just image and parent image extensions

In [4]:
df_23 = de_dupe()

 1:43AM UTC on Mar 31, 2023 --- read time: 0.3529171943664551 seconds ---


In [5]:
df_23 = df_23.cache()

In [6]:
df_23.count()

1209934

In [8]:
df_23_enc_3,dict_mapping_23_3 = trace_encode(df_23, 3,
                                   ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


found connections: 0.7539646625518799


event traces: 17373568
create graph: 22.52540636062622


transposed explode and bin: 149.23698830604553


one-hot time: 355.5194504261017


total elapsed time: 380.7928717136383


In [11]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
df_23_enc_3.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/encoded/real/23Sep3")

In [12]:
dict_mapping_23_3

{'file_path_ext': ['None',
  'empty',
  '.dll',
  'other',
  '.pyc',
  '.ini',
  '.pf',
  '.tmp',
  '.exe',
  '.bat'],
 'image_path_ext': ['.exe', 'empty', 'None', '.com'],
 'parent_path_ext': ['.exe', 'None', 'empty'],
 'object': ['PROCESS', 'FILE', 'FLOW', 'SHELL'],
 'action': ['OPEN',
  'READ',
  'MODIFY',
  'INFO',
  'CREATE',
  'WRITE',
  'DELETE',
  'RENAME',
  'COMMAND'],
 'timestamp_bins': ['4.0', '1.0', '0.0', '2.0', '3.0']}

In [9]:
df_23_enc_4,dict_mapping_23_4 = trace_encode(df_23, 4,
                                   ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action'])

found connections: 0.4708554744720459


event traces: 18438173
create graph: 25.82018232345581


23/03/31 01:57:18 ERROR TaskSetManager: Total size of serialized results of 4 tasks (1256.5 MiB) is bigger than spark.driver.maxResultSize (1024.0 MiB)


Py4JJavaError: An error occurred while calling o1229.approxQuantile.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 4 tasks (1256.5 MiB) is bigger than spark.driver.maxResultSize (1024.0 MiB)
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2607)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1182)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$hand

23/03/31 01:57:22 ERROR TaskSetManager: Total size of serialized results of 5 tasks (1590.5 MiB) is bigger than spark.driver.maxResultSize (1024.0 MiB)


In [ ]:
df_23_enc_6,dict_mapping_23_6 = trace_encode(df_23, 6,
                                   ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action'])